In [2]:
# !pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.44.0.2-py2.py3-none-any.whl size=265363806 sha256=2dec843624ca10380ffa56b7294b462d8178364662cea4cfbbf0fd7cc3dd72ad
  Stored in directory: /root/.cache/pip/wheels/e9/ea/ab/889709967692e30c57c46137edd2cae6b2768bb65ec7cb8aa5
Successfully built h2o


In [44]:
import pandas as pd
import h2o
from sklearn.model_selection import train_test_split
from h2o.automl import H2OAutoML
from sklearn.metrics import mean_squared_error

In [7]:
TOX = pd.read_excel("data.xlsx")

In [9]:
TOX.head()

,TAID,SMILES,O,Halogens,S,P,Other Metal,Hydroxyl or Ether,"Ketones, aldehydes, carboxylic acids, or esters",Which Halogens,Which Other Metal,Primary Amine or Ammonium Ion,Secondary Amine or Aromatic Nitrogen,Tertiary Amine or Amide Nitrogen,Aliphatic,Partially Aromatic,Aromatic,mouse intraperitoneal LD50,group
0,TOX-145,Oc1ccc(O)cc1,1,0,0,0,0,1,0,NaN,NaN,0,0,0,0,0,1,3.0418,3
1,TOX-245,S=P(N1CC1)(N1CC1)N1CC1,0,0,1,1,0,0,0,NaN,NaN,0,0,1,1,0,0,4.2356,5
2,TOX-1273,ClC1C(Cl)C(Cl)C(Cl)C(Cl)C1Cl,0,1,0,0,0,0,0,Cl,NaN,0,0,0,1,0,0,3.3667,4
3,TOX-1279,CN(C)C(=N)N=C(N)N,0,0,0,0,0,0,0,NaN,NaN,1,1,1,0,0,0,2.6416,3
4,TOX-1282,Cc1ccc(C(C)C)cc1O,1,0,0,0,0,1,0,NaN,NaN,0,0,0,0,0,1,3.3116,4


In [50]:
Xtrain = TOX.drop(columns=['TAID', 'SMILES', 'mouse intraperitoneal LD50'])
ytrain = TOX['mouse intraperitoneal LD50']

# Split the data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [12]:
# H2O.ai
h2o.init()

Xtrain_h2o = h2o.H2OFrame(X_train)
Xtest_h2o = h2o.H2OFrame(X_test)
X_valid_h2o = h2o.H2OFrame(X_valid)
ytrain_h2o = h2o.H2OFrame(list(y_train), column_names=["mouse intraperitoneal LD50"])
ytest_h2o = h2o.H2OFrame(list(y_test), column_names=["mouse intraperitoneal LD50"])
yvalid_h2o = h2o.H2OFrame(list(y_valid), column_names=["mouse intraperitoneal LD50"])

train_h2o = Xtrain_h2o.cbind(ytrain_h2o)
test_h2o = Xtest_h2o.cbind(ytest_h2o)
valid_h2o = X_valid_h2o.cbind(yvalid_h2o)

aml = H2OAutoML(max_runtime_secs=3600, max_models=10, seed=42, nfolds=5)
aml.train(y="mouse intraperitoneal LD50", training_frame=train_h2o, validation_frame=valid_h2o)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,26 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,2 days
H2O_cluster_name:,H2O_from_python_unknownUser_oplc3d
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.169 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
05:47:23.43: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),9/10
# GBM base models (used / total),4/4
# XGBoost base models (used / total),3/3
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [13]:
# View AutoML leaderboard
leaderboard = aml.leaderboard
print(leaderboard)

model_id                                                    rmse        mse       mae      rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20231111_54722     0.248085  0.0615463  0.176928  0.0555591                 0.0615463
StackedEnsemble_BestOfFamily_1_AutoML_1_20231111_54722  0.24844   0.0617222  0.177448  0.0556437                 0.0617222
GBM_3_AutoML_1_20231111_54722                           0.248551  0.0617778  0.176513  0.0557831                 0.0617778
GBM_4_AutoML_1_20231111_54722                           0.248694  0.0618485  0.17621   0.0557439                 0.0618485
GBM_2_AutoML_1_20231111_54722                           0.249195  0.0620982  0.17772   0.0559785                 0.0620982
GBM_1_AutoML_1_20231111_54722                           0.250751  0.0628761  0.178882  0.0560278                 0.0628761
XGBoost_3_AutoML_1_20231111_54722                       0.251055  0.0630285  0.180394  0.0563925                 0.0630285
XGBoost_2_AutoML

In [15]:
test_h2o = h2o.H2OFrame(X_test)
best_model = aml.leader

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [51]:
pred = best_model.predict(test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'Which Other Metal' has levels not trained on: ["Cd"]
  warnings.warn(w)


In [52]:
import numpy as np

y_test = y_test.to_numpy()
pred = pred.as_data_frame()
pred = pred.to_numpy()

In [53]:
mse = mean_squared_error(y_test, pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.060286582660554776


In [55]:
Xtrain = TOX.drop(columns=['TAID', 'SMILES', 'mouse intraperitoneal LD50'])
ytrain = TOX['mouse intraperitoneal LD50']

# Split the data into training (80%), and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(Xtrain, ytrain, test_size=0.2, random_state=42)

In [56]:
# One-hot coding
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
X_train = pd.get_dummies(X_train, columns=non_numeric_columns)
X_test = pd.get_dummies(X_test, columns=non_numeric_columns)
X_train, X_test = X_train.align(X_test, join='inner', axis=1)  # Align the DataFrames by the columns

In [57]:
# Gradient Boosting Machine (GBM)
from sklearn.ensemble import GradientBoostingRegressor  # For regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

gbm_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm_regressor.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [58]:
y_pred = gbm_regressor.predict(X_test)

In [59]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.06387614409802723


In [60]:
# tuning

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

gbm = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

In [61]:
y_pred = best_model.predict(X_test)

In [62]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.06240351250254019


In [63]:
# XGBoost
import xgboost as xgb

xgb_regressor = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [64]:
y_pred = xgb_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.06389423088769108


In [65]:
# tuning
xgb_model = xgb.XGBRegressor(random_state=42)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

In [66]:
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.06227256653045043


In [67]:
# Distributed Random Forest
from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [68]:
y_pred = rf_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.061631313456729925
